In [4]:
import torch
import torch.nn as nn


In [22]:
x = torch.arange(24).reshape(2, 3, 4).float()
liner = nn.Linear(in_features = 4, out_features = 12, bias = False)
liner(x).shape
 

torch.Size([2, 3, 12])

In [10]:
import torch.nn as nn


class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y

In [13]:
x_reshape = x.reshape(-1, x.size(-1))

In [16]:
x_reshape.shape

torch.Size([6, 4])

In [15]:
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [23]:
x = torch.arange(24).reshape(2, 3, 4).float()
conv = nn.Conv1d(3, 1, 2)
y = TimeDistributed(conv, True)

In [25]:
y(x)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [1, 3, 2], but got 2-dimensional input of size [6, 4] instead

In [31]:
class Flatten(nn.Module):
    # 构造函数，没有什么要做的
    def __init__(self):
        # 调用父类构造函数
        super(Flatten, self).__init__()

    # 实现forward函数
    def forward(self, input):
        # 保存batch维度，后面的维度全部压平，例如输入是28*28的特征图，压平后为784的向量
        print(input.shape)
        return input.contiguous().view(input.size(0), -1)

In [32]:
flatten = Flatten()
y = flatten(x)

torch.Size([2, 3, 4])


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
        [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.]])

In [35]:

x = torch.arange(24).reshape(2, 3, 4).float()
x.contiguous().view(x.size(0), -1)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
        [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.]])

In [36]:
x.size(0)

2

In [39]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[-2.4194, -1.3630, -1.3102,  1.1967],
        [ 0.4259, -2.6712,  0.1803,  0.3913],
        [-0.3938,  2.9129, -0.8097, -6.5617]])

In [51]:
X = torch.ones(4, 3, 128)
lstm = nn.LSTM(128, 256, batch_first = True)

In [52]:
a, b = lstm(X)

In [53]:
a.shape

torch.Size([4, 3, 256])

In [54]:
b[0].shape

torch.Size([1, 4, 256])

In [55]:
b[1].shape

torch.Size([1, 4, 256])

In [1]:
import torch
import torch.nn as nn


rnn = nn.LSTM(input_size=10, hidden_size=20, num_layers=2, batch_first=True)
# 输入数据x的向量维数10, 设定lstm隐藏层的特征维度20, 此model用2个lstm层。如果是1，可以省略，默认为1)


input = torch.randn(3, 5, 10)

# 输入的input为，序列长度seq_len=5, 每次取的minibatch大小，batch_size=3, 数据向量维数=10（仍然为x的维度）。每次运行时取3个含有5个字的句子（且句子中每个字的维度为10进行运行）


# 初始化的隐藏元和记忆元,通常它们的维度是一样的

# 2个LSTM层，batch_size=3, 隐藏层的特征维度20

h0 = torch.randn(3, 2, 20)

c0 = torch.randn(3, 2, 20)


# 这里有2层lstm，output是最后一层lstm的每个词向量对应隐藏层的输出,其与层数无关，只与序列长度相关

# hn,cn是所有层最后一个隐藏元和记忆元的输出

output, (hn, cn) = rnn(input)

##模型的三个输入与三个输出。三个输入与输出的理解见上三输入，三输出

In [2]:
output.shape

torch.Size([3, 5, 20])

In [26]:
import os
os.environ['KERAS_BACKEND']='tensorflow'
from keras.layers import Dense, LSTM
# from keras.layers import Input, Dropout, Activation
# from keras.layers import Convolution2D, MaxPooling2D, Reshape
from keras.layers import Convolution1D, MaxPooling1D, Flatten
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed #输入至少为3D张量
# from matplotlib import pyplot as plt
# import random
import argparse
import time
import numpy as np

In [50]:
X = np.ones((2, 3, 10, 48))

X.shape

(2, 3, 10, 48)

In [51]:
def model_fit(train_X):
    # define model
    model = Sequential()
    # model.add(TimeDistributed(cnn))
    model.add(TimeDistributed(Convolution1D(128, 4, border_mode='same'), input_shape=train_X.shape[1:]))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128, return_sequences=True, name="lstm_layer0")) #output = batch * step * output_size
    model.add(LSTM(128, return_sequences=False, name="lstm_layer1")) #output = batch * output_size(last step of every batch)
    # model.add(LSTM(100, return_sequences=True, name="lstm_layer2"))
    model.add(Dense(1, activation='sigmoid'))
    # model.add(GlobalAveragePooling1D(name="global_avg"))   

    predict_y = model.predict(test_X)




In [78]:
X = np.ones((2, 3, 10, 48))



model = Sequential()
model.add(TimeDistributed(Convolution1D(128, 4, border_mode='same'), input_shape=X.shape[1:]))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
#model.add(TimeDistributed(Flatten()))

#model.add(LSTM(128, return_sequences=True, name="lstm_layer0")) #output = batch * step * output_size
#model.add(LSTM(128, return_sequences=False, name="lstm_layer1")) #output = batch * output_size(last step of every batch)
# model.add(LSTM(100, return_sequences=True, name="lstm_layer2"))
#model.add(Dense(1, activation='sigmoid'))
#model.add(GlobalAveragePooling1D(name="global_avg"))   

In [79]:
model.predict(X).shape

(2, 3, 5, 128)

In [48]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 10, 48)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 480)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [2]:
import random
import torch
input = []
lable = []
for i in range(100):
    input.append(torch.randn(10, 48))
    lable.append(1)
a = range(1, 100, 1)
b = random.sample(a, 50)
lable[b] = 0

TypeError: list indices must be integers or slices, not list